## Storage reuse with multiple datasets

#### This notebook describes a storage reuse scenario with multiple datasets containing common datapoints.

When the same file is used in multiple datasets, that file will be added to the bucket only once, in order to optimize the space usage in the bucket. To exemplify this use case, two entities will be created: the people entity contains 10 images with faces of people, while famous entity contains 7 images with faces of famous people, being 5 of them also contained in the people entity.

This way, when sending the files to the repository, the 5 images that are being used in the two entities will not be duplicated in the bucket. The two entities will refer to the same image stored in the bucket.

#### To start using the ml-git api we need to import it into our script

In [1]:
from ml_git import api

#### After that, we define some variables that will be used by the notebook

In [2]:
# The type of entity we are working on
entity = 'datasets'

# The entity name we are working on
entity_name_people = 'people_faces'

# The entity name we are working on
entity_name_famous = 'famous_faces'

#### To start, let's take into account that you have a repository with git settings to make the clone. If this is not your scenario, you will need to configure ml-git outside this notebook (At the moment the api does not have the necessary methods to perform this configuration). 

#### Or you can manually configure the repository using the command line, following the steps in the [First Project](https://github.com/HPInc/ml-git/blob/development/docs/first_project.md) documentation.

In [3]:
repository_url = '/local_ml_git_config_server.git'

api.clone(repository_url)

INFO - Metadata Manager: Metadata init [/local_server.git/] @ [/api_scripts/multiple_datasets_notebook/tmpfyoxmtjy/mlgit/.ml-git/datasets/metadata]
INFO - Metadata Manager: Metadata init [/local_server.git/] @ [/api_scripts/multiple_datasets_notebook/tmpfyoxmtjy/mlgit/.ml-git/models/metadata]
INFO - Metadata Manager: Metadata init [/local_server.git/] @ [/api_scripts/multiple_datasets_notebook/tmpfyoxmtjy/mlgit/.ml-git/labels/metadata]
INFO - Metadata: Successfully loaded configuration files!


#### Create the people dataset

![dataset](./people_faces/people_faces.jpg)

In [4]:
!ml-git datasets create people_faces --categories="computer-vision, images" --bucket-name=faces_bucket --mutability=strict --import='people_faces' --unzip

⠋ Importing files⠙ Importing filesINFO - MLGit: Unzipping files
INFO - MLGit: Dataset artifact created.


#### We can now proceed with the necessary steps to send the new data to storage.

In [5]:
api.add(entity, entity_name_people, bumpversion=True)

INFO - Metadata Manager: Pull [/api_scripts/multiple_datasets_notebook/.ml-git/datasets/metadata]
INFO - Repository: datasets adding path [/api_scripts/multiple_datasets_notebook/datasets/people_faces] to ml-git index
files: 100%|██████████| 10.0/10.0 [00:00<00:00, 646files/s]


files: 100%|██████████| 10.0/10.0 [00:00<00:00, 32.7kfiles/s]


Commit the changes

In [6]:
# Custom commit message
message = 'Commit example'

api.commit(entity, entity_name_people, message)

INFO - Metadata Manager: Commit repo[/api_scripts/multiple_datasets_notebook/.ml-git/datasets/metadata] --- file[people_faces]


#### As we are using MinIO locally to store the data in the bucket, we were able to check the number of files that are in the local bucket.

In [7]:
import os

def get_bucket_files_count():
  print("Number of files on bucket: " +  str(len(os.listdir('../../data/faces_bucket'))))

#### Amount of files in the buket before pushing the people dataset

In [8]:
get_bucket_files_count()

Number of files on bucket: 0


As we have not yet uploaded any version of our dataset, the bucket is empty.

#### Pushing the people dataset

In [9]:
api.push(entity, entity_name_people)

files: 100%|██████████| 20.0/20.0 [00:00<00:00, 92.3files/s]


⠋ Pushing metadata to the git repository⠙ Pushing metadata to the git repository

#### Amount of files in the buket after pushing the people dataset

In [10]:
get_bucket_files_count()

Number of files on bucket: 20


After sending the data, we can observe the presence of 20 blobs related to the 10 images that were versioned. In this case, two blobs were added for each image in our dataset.

#### Create the famous dataset

Let's create our second dataset that has some images equals to the first dataset.

![dataset](famous_faces/famous_faces.jpg)

In [11]:
!ml-git datasets create famous_faces --categories="computer-vision, images" --bucket-name=faces_bucket --mutability=strict --import='famous_faces' --unzip

⠋ Importing files⠙ Importing filesINFO - MLGit: Unzipping files
INFO - MLGit: Dataset artifact created.


#### We can now proceed with the necessary steps to send the new data to storage.

In [12]:
api.add(entity, entity_name_famous, bumpversion=True)

INFO - Metadata Manager: Pull [/api_scripts/multiple_datasets_notebook/.ml-git/datasets/metadata]
INFO - Repository: datasets adding path [/api_scripts/multiple_datasets_notebook/datasets/famous_faces] to ml-git index
files: 100%|██████████| 7.00/7.00 [00:00<00:00, 703files/s]


files: 100%|██████████| 7.00/7.00 [00:00<00:00, 40.2kfiles/s]


Commit the changes

In [13]:
# Custom commit message
message = 'Commit example'

api.commit(entity, entity_name_famous, message)

INFO - Metadata Manager: Commit repo[/api_scripts/multiple_datasets_notebook/.ml-git/datasets/metadata] --- file[famous_faces]


And finally, sending the data

In [14]:
api.push(entity, entity_name_famous)

files: 100%|██████████| 14.0/14.0 [00:00<00:00, 177files/s]


⠋ Pushing metadata to the git repository⠙ Pushing metadata to the git repository

#### Amount of files in the buket after pushing the famous dataset

In [15]:
get_bucket_files_count()

Number of files on bucket: 24


As you can see, only 4 blobs were added to our bucket. Of the set of 7 images, only 2 images were different from the other dataset, so ml-git can optimize storage by adding blobs related only to these new images.